In [1]:
!pip install confluent_kafka
!pip install pymongo

In [2]:
from confluent_kafka import Consumer
import pymongo

In [15]:
def login_info():
    info={
        'bootstrap.servers':'xxxxxxxxxxxxxxxxxxxxxxxxxxx',
        'security.protocol':'SASL_SSL',
        'sasl.mechanism':'PLAIN',
        'sasl.username':'xxxxxxxxxxxxxxxxx',
        'sasl.password':'xxxxxxxxxxxxxxxxxxxxxxxxxxx',
        'partitioner':'consistent_random',
        'group.id':'testing',
        'auto.offset.reset':'earliest'

    }
    return info

In [16]:
c=Consumer(login_info())

In [17]:
print(c.list_topics().topics)
d=c.list_topics().topics

{'topic_1': TopicMetadata(topic_1, 3 partitions), 'test': TopicMetadata(test, 3 partitions)}


In [18]:
topic='topic_1'
c.subscribe([topic])

In [4]:
client=pymongo.MongoClient('xxxxxxxxxxxxxxxxxxxxxxx')

In [5]:
Database='Realtime'
collection='sql_to_nosql'

In [7]:
db=client[Database]
col=db[collection]

Above are the connectivity part in which made connectivity to Kafka server and subscribed to topic:topic_1 and made MongoDB connection, connected to 'Realtime' Database 'sql_to_nosql' collection

Below we are pulling the records from kafka topic inserting it into sql_to_nosql collection to test collection which will not load duplicates

In [19]:
while True:
    try:
        msg=c.poll(5.0)
        
        if msg is None:
            print('wainting for messages!!')
            continue
        elif msg is not None:
            msg_value=msg.value().decode('utf-8')
            #print(msg_value)
            print(type(msg_value))
            y=eval(msg_value)
            t=col.insert_one(y)
            print(f'The following messages are f{msg.value()} which was delivered')
            
    except KeyboardInterrupt:
        print('closing the consumer code')
        break

agg=col.aggregate(
    [
        {
            '$group':
            {
                '_id':{'ID':'$ID','Name':'$Name','CountryCode':'$CountryCode','District':'$District','Population':'$Population'},
                
            }
        },
        {
            '$out':'test'
        }
    ]
)
c.close()

<class 'str'>
The following messages are fb"{'ID': 4, 'Name': 'Mazar-e-Sharif', 'CountryCode': 'AFG', 'District': 'Balkh', 'Population': 127800}" which was delivered
<class 'str'>
The following messages are fb"{'ID': 5, 'Name': 'Amsterdam', 'CountryCode': 'NLD', 'District': 'Noord-Holland', 'Population': 731200}" which was delivered
<class 'str'>
The following messages are fb"{'ID': 6, 'Name': 'Rotterdam', 'CountryCode': 'NLD', 'District': 'Zuid-Holland', 'Population': 593321}" which was delivered
<class 'str'>
The following messages are fb"{'ID': 10, 'Name': 'Tilburg', 'CountryCode': 'NLD', 'District': 'Noord-Brabant', 'Population': 193238}" which was delivered
<class 'str'>
The following messages are fb"{'ID': 12, 'Name': 'Breda', 'CountryCode': 'NLD', 'District': 'Noord-Brabant', 'Population': 160398}" which was delivered
<class 'str'>
The following messages are fb"{'ID': 15, 'Name': 'Enschede', 'CountryCode': 'NLD', 'District': 'Overijssel', 'Population': 149544}" which was deliver